# Lung Detection Final Project
Danny Mathieson

### Project Contents

##### [Project Setup](#project-setup)
1. [Import Necessary Libraries](#1-import-necessary-libraries)
2. [Plot sample images for all classes](#2-plot-sample-images-for-all-classes)
3. Plot the distribution of images across classes
4. Build Data Augmentation for the training data with translation, rescale, and flip - Rescale images to 48x48
5. Build Data Augmentation for the test data with translation, rescale, and flip - Rescale images to 48x48
6. Make funciton to read directly from the train and test folders

##### Build Initial CNN
1. Build a CNN with different filters, max pooling, dropout, and batch normalization layers
2. Use ReLU as an activation function
3. Use Categorical Cross Entropy as a loss function
4. Use rmsprop as the optimizer
5. Use Early stopping with a patience of 2 epochs on validation loss or validation accuracy
6. Use 10 epochs
7. Train using a generator and test the accuracy on the test data at each epoch
8. Plot training & validation accuracy & loss
9. Observe Precision, Recallm F1-Score for all classes on both grayscale & color models - determine if the classes are good.

##### Transfer Learning - Mobile Net
1. Prepare the dataset for the mobile-net model with color mode RGB
2. Create an instance of the mobile-net pre-trained model
3. Add a dense layer, dropout layer, and batch normalizatiopn layer on the pre-trained model
4. Create a final output using the softmax activation function
5. Change the batch size activation function and optimize as rmsprop - observe if the accuracy increases
6. Change the loss function to categorical cross-entropy
7. Use Early Stopping on the validation loss with a patience of 2 epochs to prevent overfitting
8. Use 10 epochs
9. Train using a generator and test the accuracy on the test data at each epoch
10. Plot training & validation accuracy & loss
11. Observe Precision, Recallm F1-Score for all classes on both grayscale & color models - determine if the classes are good.

##### Transfer Learning - Densenet121
1. Prepare the dataset for the densenet121 model with image size 224x224x3
2. Freeze the top layers of the pre-trained model
3. Add a dense layer at the end of the pre-trained model, followed by a dropout layer and try various combinations to optimize accuracy
4. Create a final output using the softmax activation function
5. Change the loss function to categorical cross-entropy
6. Use Adam as the optimizer
7. Use Early Stopping on the validation loss with a patience of 2 epochs to prevent overfitting
8. Use 15 epochs with a batch size of 7 - tinker with these to optimize accuracy
9. Train using a generator and test the accuracy on the test data at each epoch
10. Plot training & validation accuracy & loss
11. Observe Precision, Recallm F1-Score for all classes on both grayscale & color models - determine if the classes are good.

##### Final Step
1. Compare all of the models on the basis of accuracy, precision, recall, f1-score

### Project Setup

##### 1. Import Necessary Libraries

In [1]:
import os
import shutil
import numpy as np
from tensorflow.keras.utils import load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2023-03-28 15:52:25.757735: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
shutil.unpack_archive('./images/Dataset_Detection_of_Lung_Infection.zip', './images')

##### 2. Plot sample images for all classes

In [ ]:
classes = ['Healthy', 'Type 1 disease', 'Type 2 disease']
train_path = './images/data/train/'